<a href="https://colab.research.google.com/github/tusker4/Sesac_Saltlux_DeepLearning/blob/main/8_1_NLP_koBert%EA%B8%B0%EB%B0%98_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98_TPU_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D2_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 로드

In [2]:
import numpy as np

In [3]:
loaded_data = np.load('/content/drive/MyDrive/머신러닝(솔트룩스)/딥 러닝/딥러닝_NLP/categorial.npy')

In [4]:
loaded_data['X_train'].shape, loaded_data['y_val'].shape, loaded_data['y_test'].shape

((3, 941814, 128), (2490,), (5010,))

In [5]:
# 데이터 받기
X_train = loaded_data['X_train']
y_train = loaded_data['y_train']

X_val   = loaded_data['X_val']
y_val   = loaded_data['y_val']

X_test  = loaded_data['X_test']
y_test  = loaded_data['y_test']

In [6]:
# 가비지 컬렉션
import gc

gc.collect()


99

# 사전학습된 모델 로드(기존모델구조+추가(3개로 분류되는 레이어 추가)

In [7]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.6 MB/s eta 0:00:00


In [8]:
from transformers import BertTokenizer, TFBertModel

repo = 'klue/bert-base'
tokenizer = BertTokenizer.from_pretrained( repo )

In [9]:
X_train.reshape(-1, 3, 128).shape

(941814, 3, 128)

In [10]:
# (데이터, 3가지정보, 문장1개당 토큰수)
X_train = X_train.reshape( -1, 3, 128)
X_train = X_val.reshape( -1, 3, 128)
X_train = X_test.reshape( -1, 3, 128)

## 출력이 3개(다중) 분류하는 층을 추가

In [13]:
import tensorflow as tf

# 기존모델(koBert계열) + 출력층 추가
class MultiClfModel(tf.keras.Model):
  # 생성자
  def __init__(self, repo_name, label_num):
    '''
        - repo_name : 허깅페이스에서 제공하는 기존 모델의 저장소 주소
        - label_num : 새로 붙일 출력층의 분류개수
    '''
    super(MultiClfModel, self).__init__()  # 부모 생성자 명시적 호출
    # 재료(신경망, 레이어) 준비
    self.kobert = TFBertModel.from_pretrained( repo_name, from_pt =True)
    self.cut_output = tf.keras.layers.Dense(label_num, activation= 'softmax', name='clf',
                                        # 정규분포값을 조정해서 -> 가중치 초기값을 부여 -> 성능, 학습속도 효율성 극대화
                                        # 평균=0.01은 설정값
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(0.01))

    pass

  # 순전파시, 커스텀 위해 call함수 재정의 (부모에게 받은 함수)
  def call(self, inputs):
    # 1. 입력 데이터에서 3파트를 분리
    # input_ids      : 문장의 백터화 처리  = 스페셜토큰 + 문장1 + 스페셜토큰 + 문장2 + 스페셜토큰 + 패딩값
    # token_type_ids : 문장1=0, 문장2=1, 패딩=0
    # attention_mask : 1=문장, 0=패딩

    input_ids      = inputs[ : , 0, : ] # 문장 각각은 128토큰으로 구성됨
    token_type_ids = inputs[ : , 1, : ] # 메타정보, 앞문장, 뒷문장
    attention_mask = inputs[ : , 2, : ] # 메타정보 문장(어텐션 집중과패딩)
    layer = self.kobert(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask=attention_mask, )
    # bert의 1번 자리를 출력에 연결
    output = self.cut_output( layer[1] )
    return output
  pass

model = MultiClfModel(repo, 3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [14]:
# 모델 컴파일
# SparseCategoricalCrossentropy : 다중분류, 정답이 정수
# CategoricalCrossentropy : 다중분류, 정답이 부동소수, 원핫인코딩, 원래값
# learning_rate=0.001 기본값을 설정, 차후 수정 가능
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss     = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics  = ['accuracy'],
              )


# TPU 학습

In [22]:
strategy = None

In [23]:
import tensorflow as tf
# TPU 사용설정
try:
  # TPU 클러스터에 접근하여 TPU를 엑세스하는 세션 획득
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
  # 등록
  tf.config.experimental_connect_to_cluster( resolver )
  tf.tpu.experimental.initialize_tpu_system( resolver )
  # TPU 영역 표시
  strategy = tf.distribute.experimental.TPUStrategy( resolver )
  pass
except Exception as e:

  print('TPU 지원불가')

TPU 지원불가


In [24]:
# 조기학습 종료
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2)


In [25]:
# TPU 학습
if strategy:
  with strategy.scope():
    model.fit(X_train, y_train, epochs=1, batch_size=32,
              validation_data=(X_val, y_val), callbacks=[early_stopping])
else:
  print('cpu 학습')

cpu 학습


# 테스트